# NBA Games Preprocessor

This notebook takes games and transforms them into richer datasets.  This takes a reasonable amount of processing so this work is done upfront and commited to CSV.

## Configuration

In [1]:
input_filename = '../input/games.json'
output_parsed_data_filename = '../input/parsed_data.csv'

#input_filename = '../input/current_games.json'
#output_parsed_data_filename = '../input/parsed_data_current_games.csv'

input_odds_filename = '../results/odds/odds.csv'

use_team_hot_encoding = False
use_team_home_away_hot_encoding = False

# Processor

In [2]:
import json 
import pandas as pd
import numpy as np 
import geopy.distance

#package for flattening json in pandas df
from pandas.io.json import json_normalize

In [3]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Load data from file system

In [4]:
with open(input_filename) as gamesJsonRaw:
    d = json.load(gamesJsonRaw)

games = json_normalize(d['games'])

In [5]:
len(games)

905

In [6]:
games.sample(5)

,id,location,scoreAway,scoreHome,scoreQuarters,season,statsAway.Ast,statsAway.Blk,statsAway.BlkAgainst,statsAway.DefReb,...,statsHome.Reb,statsHome.Stl,statsHome.Tov,statsHome.WinPct,statsHome.Wins,teamAwayCode,teamAwayId,teamHomeCode,teamHomeId,time
418,42496,Pepsi Center,111,117,"[{'number': 1, 'scoreHome': 20, 'scoreAway': 3...",2017,27,8,4,41,...,57,12,15,1.000,1,NOP,110,DEN,99,2017-12-15T21:00:00.000Z
726,42806,AT&T Center,98,113,"[{'number': 1, 'scoreHome': 29, 'scoreAway': 2...",2017,21,3,2,31,...,59,4,7,1.000,1,SAC,103,SAS,106,2018-01-28T19:00:00.000Z
887,42968,Barclays Center,87,104,"[{'number': 1, 'scoreHome': 28, 'scoreAway': 1...",2017,17,7,9,42,...,53,4,15,1.000,1,CHI,89,BRO,84,2018-02-26T19:30:00.000Z
402,42480,Talking Stick Resort Arena,115,109,"[{'number': 1, 'scoreHome': 31, 'scoreAway': 3...",2017,16,3,9,38,...,51,3,11,0.000,0,TOR,81,PHX,104,2017-12-13T21:00:00.000Z
177,42252,AT&T Center,94,133,"[{'number': 1, 'scoreHome': 37, 'scoreAway': 1...",2017,22,4,6,26,...,53,5,13,1.000,1,CHI,89,SAS,106,2017-11-11T20:30:00.000Z


In [7]:
#load stadiums - json object
with open('../input/stadiums.json') as stadiumsJsonRaw:
    stadiumsJson = json.load(stadiumsJsonRaw)

stadiums = json_normalize(stadiumsJson['stadiums'])

In [8]:
stadiums.head(5)

,lat,lng,team
0,33.757183,-84.396278,ATL
1,42.366281,-71.062266,BOS
2,40.682650,-73.974689,BRO
3,35.224519,-80.841053,CHA
4,41.880589,-87.674149,CHI


## Set Data Types

In [9]:
games['season'] = games['season'].astype('int')
games['scoreAway'] = games['scoreAway'].astype('uint8')
games['scoreHome'] = games['scoreHome'].astype('uint8')

games['statsHome.FgAtt'] = games['statsHome.FgAtt'].astype('uint8')
games['statsHome.OffReb'] = games['statsHome.OffReb'].astype('uint8')
games['statsHome.Tov'] = games['statsHome.Tov'].astype('uint8')
games['statsHome.FtAtt'] = games['statsHome.FtAtt'].astype('uint8')
games['statsHome.Pts'] = games['statsHome.Pts'].astype('uint8')
games['statsHome.PtsAgainst'] = games['statsHome.PtsAgainst'].astype('uint8')

games['statsAway.FgAtt'] = games['statsAway.FgAtt'].astype('uint8')
games['statsAway.OffReb'] = games['statsAway.OffReb'].astype('uint8')
games['statsAway.Tov'] = games['statsAway.Tov'].astype('uint8')
games['statsAway.FtAtt'] = games['statsAway.FtAtt'].astype('uint8')
games['statsAway.Pts'] = games['statsAway.Pts'].astype('uint8')
games['statsAway.PtsAgainst'] = games['statsAway.PtsAgainst'].astype('uint8')

#scoreQuarters          object
#teamAwayCode           object
#teamHomeCode           object

In [10]:
games.dtypes

id                           int64
location                    object
scoreAway                    uint8
scoreHome                    uint8
scoreQuarters               object
season                       int64
statsAway.Ast               object
statsAway.Blk               object
statsAway.BlkAgainst        object
statsAway.DefReb            object
statsAway.Ejections         object
statsAway.Fg2PtAtt          object
statsAway.Fg2PtMade         object
statsAway.Fg2PtPct          object
statsAway.Fg3PtAtt          object
statsAway.Fg3PtMade         object
statsAway.Fg3PtPct          object
statsAway.FgAtt              uint8
statsAway.FgMade            object
statsAway.FgPct             object
statsAway.FoulFlag1         object
statsAway.FoulFlag1Drawn    object
statsAway.FoulFlag2         object
statsAway.FoulFlag2Drawn    object
statsAway.FoulPers          object
statsAway.FoulPersDrawn     object
statsAway.FoulTech          object
statsAway.FoulTechDrawn     object
statsAway.Fouls     

## Match the odds to the games

In [11]:
odds = pd.read_csv(input_odds_filename)
odds = odds.rename(columns={'HomeCode': 'teamHomeCode', 'AwayCode': 'teamAwayCode'})
len(odds)

11800

In [12]:
odds.sample(5)

,YY,DD,MM,Time,Country,League,Season,HomeTeam,AwayTeam,teamHomeCode,teamAwayCode,HomeScore,AwayScore,MatchbookHomeOdd,MatchbookAwayOdd,PinnacleHomeOdd,PinnacleAwayOdd,Bet365HomeOdd,Bet365AwayOdd
6833,2013,22,Mar,11:00:00 PM,USA,NBA,2012/2013,Indiana Pacers,Milwaukee Bucks,IND,MIL,102.0,78,NaN,NaN,1.33,3.69,1.31,3.60
10273,2010,10,Nov,03:00:00 AM,USA,NBA,2010/2011,Portland Trail Blazers,Detroit Pistons,POR,DET,100.0,78,NaN,NaN,1.15,6.27,1.14,5.75
2189,2016,21,Oct,12:00:00 AM,USA,NBA,2016/2017,Orlando Magic,New Orleans Pelicans,ORL,NOP,114.0,111,1.47,2.57,1.51,2.65,1.47,2.75
1795,2016,15,Dec,01:00:00 AM,USA,NBA,2016/2017,Houston Rockets,Sacramento Kings,HOU,SAC,132.0,98,1.22,5.10,1.21,5.00,1.18,5.25
4924,2014,9,Nov,12:00:00 AM,USA,NBA,2014/2015,Indiana Pacers,Washington Wizards,IND,WAS,90.0,97,2.39,1.59,2.46,1.62,2.40,1.62


In [13]:
# Fix issue with missing data
odds.iloc[4163, odds.columns.get_loc('MM')] = 'Feb'

# Make sure all are valid month keys
odds['MM'].unique()

array(['Feb', 'Jan', 'Dec', 'Nov', 'Oct', 'Jun', 'May', 'Apr', 'Mar'], dtype=object)

#### Odds: Convert Date to YYYYMMDD

- Do two dates as the days can offset by 1 day

In [14]:
import calendar
monthMap = dict((v,k) for k,v in enumerate(calendar.month_abbr))

def getAsDoubleDigit(x):
    return str(x).zfill(2)

def getMonth(x):
    m = monthMap[x]
    return getAsDoubleDigit(m)

odds['MM_Code'] = odds['MM'].apply(lambda x: getMonth(x))
odds['DD_Code'] = odds['DD'].apply(lambda x: getAsDoubleDigit(x))

odds['day'] = odds['YY'].map(str) + odds['MM_Code'] + odds['DD_Code']
odds['day2'] = odds['YY'].map(str) + odds['MM_Code'] + odds['DD_Code']

#### Games: Convert Date to YYYYMMDD

- Do two dates as the days can offset by 1 day

In [15]:
# Convert time to datetime data type
games['time'] =  pd.to_datetime(games['time'])
games['time2'] =  games['time'] + pd.Timedelta(hours=24)

# Convert date to YYYYMMDDpd.merge(a, b, on=['A', 'B'])
games['day'] = games['time'].dt.year.map(str) + games['time'].dt.month.map(lambda x: str(x).zfill(2)) + games['time'].dt.day.map(lambda x: str(x).zfill(2))
games['day2'] = games['time2'].dt.year.map(str) + games['time2'].dt.month.map(lambda x: str(x).zfill(2)) + games['time2'].dt.day.map(lambda x: str(x).zfill(2))

In [16]:
games1 = games[['day', 'teamHomeCode', 'teamAwayCode']].copy()

In [17]:
odds1 = odds[['day', 'teamHomeCode', 'teamAwayCode', 'MatchbookHomeOdd', 'MatchbookAwayOdd', 'PinnacleHomeOdd', 'PinnacleAwayOdd', 'Bet365HomeOdd', 'Bet365AwayOdd']].copy()

In [18]:
games1 = pd.merge(games1, odds1, on=['day', 'teamHomeCode', 'teamAwayCode'], how='left')

In [19]:
games2 = games[['day2', 'teamHomeCode', 'teamAwayCode']].copy()

In [20]:
odds2 = odds[['day2', 'teamHomeCode', 'teamAwayCode', 'MatchbookHomeOdd', 'MatchbookAwayOdd', 'PinnacleHomeOdd', 'PinnacleAwayOdd', 'Bet365HomeOdd', 'Bet365AwayOdd']].copy()

In [21]:
games2 = pd.merge(games2, odds2, on=['day2', 'teamHomeCode', 'teamAwayCode'], how='left')

In [22]:
games2 = games2.rename(columns={'day2': 'day'})

In [23]:
data = games1.combine_first(games2)

In [24]:
data.sample(5)

,day,teamHomeCode,teamAwayCode,MatchbookHomeOdd,MatchbookAwayOdd,PinnacleHomeOdd,PinnacleAwayOdd,Bet365HomeOdd,Bet365AwayOdd
33,20171021,MIL,POR,1.69,2.35,1.68,2.33,1.62,2.40
484,20171225,GSW,CLE,1.56,2.67,1.52,2.73,1.50,2.70
76,20171028,CHI,OKL,NaN,NaN,NaN,NaN,NaN,NaN
586,20180108,LAC,ATL,1.38,3.14,1.40,3.22,1.38,3.15
143,20171107,IND,NOP,1.85,2.05,1.91,2.00,1.86,1.95


In [25]:
data = data.drop(['day', 'teamHomeCode', 'teamAwayCode'], axis=1)

In [26]:
games = pd.merge(games, data, left_index=True, right_index=True)

In [27]:
games[['time', 'teamHomeCode', 'teamAwayCode', 'MatchbookHomeOdd', 'MatchbookAwayOdd', 'PinnacleHomeOdd', 'PinnacleAwayOdd', 'Bet365HomeOdd', 'Bet365AwayOdd']].sample(5)

,time,teamHomeCode,teamAwayCode,MatchbookHomeOdd,MatchbookAwayOdd,PinnacleHomeOdd,PinnacleAwayOdd,Bet365HomeOdd,Bet365AwayOdd
636,2018-01-15 21:00:00,UTA,IND,NaN,NaN,NaN,NaN,NaN,NaN
453,2017-12-20 21:00:00,DEN,MIN,NaN,NaN,NaN,NaN,NaN,NaN
349,2017-12-06 20:00:00,CHA,GSW,1.68,2.38,1.65,2.39,1.66,2.30
45,2017-10-23 20:30:00,SAS,TOR,1.69,2.35,1.74,2.23,1.68,2.25
308,2017-11-30 22:00:00,POR,MIL,1.69,2.35,1.69,2.30,1.68,2.25


   ## Advanced Statistic Generation

#### Remember the stats columns - these need to be removed once used.

In [28]:
statsColumns = pd.Series(list(games.columns.values))
statsColumns = statsColumns.loc[statsColumns.str.startswith(('statsAway', 'statsHome'), na=False)]

In [29]:
# Replace any NaN with 0 in the stats - games with 0 stats are NaN.
games[statsColumns] = games[statsColumns].fillna(value=0)

### Calculate the number of games played for season

In [30]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(games)), columns=['gamesPlayedHome', 'gamesPlayedAway'])

# Iterate through every team
for team in games.teamAwayCode.unique():
    season = 0

    # Iterate through each game the team is present in.
    for index, game in games[(games['teamAwayCode'] == team) | (games['teamHomeCode'] == team)].sort_values(by='season', ascending=True).iterrows():
        if season != game.season:
            gameCount = 0
            homeGameCount = 0
            season = game.season

        gameCount += 1

        # Update game count for team - whether away or home.
        if game.teamAwayCode == team:
            data.loc[index]['gamesPlayedAway'] = gameCount
        else:
            data.loc[index]['gamesPlayedHome'] = gameCount

data['gamesPlayedAway'] = data['gamesPlayedAway'].astype('uint8')
data['gamesPlayedHome'] = data['gamesPlayedHome'].astype('uint8')

# Append the results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)

In [31]:
games[['teamHomeCode', 'teamAwayCode', 'season', 'gamesPlayedHome', 'gamesPlayedAway']].sample(5)

,teamHomeCode,teamAwayCode,season,gamesPlayedHome,gamesPlayedAway
69,ORL,SAS,2017,35,36
871,LAL,DAL,2017,24,27
148,SAS,LAC,2017,42,39
695,DAL,HOU,2017,16,13
142,GSW,MIA,2017,42,39


## Calculate the Win/Loss % for the Home team.

In [32]:
def calc_percentage(x, y):
    if x == 0: return 0
    if x is None: return 0
    if y is None: return 0
    return 0 if y == 0 else (x / y)

In [33]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(
    index=range(0,len(games)), 
    columns=[
        'totalGamesHome', 'totalWinsHome', 'homeGamesHome', 'homeWinsHome', 'awayGamesHome', 'awayWinsHome',
        'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome',
        'totalGamesAway', 'totalWinsAway', 'homeGamesAway', 'homeWinsAway', 'awayGamesAway', 'awayWinsAway',
        'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway'
    ]
)

# Iterate through each game.
for index, game in games.iterrows():

    # Home Team
    # ---------

    historicGames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamHomeCode'] == game.teamHomeCode) | (games['teamAwayCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=True)

    homeGamesHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode'])])        
    homeWinsHome = len(historicGames[(historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"])])

    awayGamesHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode'])])        
    awayWinsHome = len(historicGames[(historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])])

    #totalGames = len(historicGames);
    #totalWins = len(
    #    historicGames[
    #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
    #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
    #    ]
    #)

    totalGamesHome = homeGamesHome + awayGamesHome
    totalWinsHome = homeWinsHome + awayWinsHome

    percentageTotalWinHome = round(calc_percentage(totalWinsHome, totalGamesHome), 3)
    percentageHomeWinHome = round(calc_percentage(homeWinsHome, homeGamesHome), 3)
    percentageAwayWinHome = round(calc_percentage(awayWinsHome, awayGamesHome), 3)

    # print('HOME', game['gamesPlayedHome'], 'TG', totalGamesHome, 'W', totalWinsHome, 'HGP', homeGamesHome, 'HW', homeWinsHome, 'AGP', awayGamesHome, 'AW', awayWinsHome)
    # print('HOME', 'W%', percentageTotalWinHome, 'HW%', percentageHomeWinHome, 'AW%', percentageAwayWinHome)

    # Away Team
    # ---------

    historicGames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=True)

    homeGamesAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode'])])        
    homeWinsAway = len(historicGames[(historicGames['teamHomeCode'] == game['teamAwayCode']) & (games["scoreAway"] < games["scoreHome"])])

    awayGamesAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode'])])        
    awayWinsAway = len(historicGames[(historicGames['teamAwayCode'] == game['teamAwayCode']) & (games["scoreAway"] > games["scoreHome"])])

    #totalGames = len(historicGames);
    #totalWins = len(
    #    historicGames[
    #        (historicGames['teamHomeCode'] == game['teamHomeCode']) & (games["scoreAway"] < games["scoreHome"]) |
    #        (historicGames['teamAwayCode'] == game['teamHomeCode']) & (games["scoreAway"] > games["scoreHome"])
    #    ]
    #)

    totalGamesAway = homeGamesAway + awayGamesAway
    totalWinsAway = homeWinsAway + awayWinsAway

    percentageTotalWinAway = round(calc_percentage(totalWinsAway, totalGamesAway), 3)
    percentageHomeWinAway = round(calc_percentage(homeWinsAway, homeGamesAway), 3)
    percentageAwayWinAway = round(calc_percentage(awayWinsAway, awayGamesAway), 3)

    # print('AWAY', game['gamesPlayedAway'], 'TG', totalGamesAway, 'W', totalWinsAway, 'HGP', homeGamesAway, 'HW', homeWinsAway, 'AGP', awayGamesAway, 'AW', awayWinsAway)
    # print('AWAY', 'W%', percentageTotalWinAway, 'HW%', percentageHomeWinAway, 'AW%', percentageAwayWinAway)

    # ---------    

    # Update the row with the history
    data.loc[index] = [
        totalGamesHome, totalWinsHome, homeGamesHome, homeWinsHome, awayGamesHome, awayWinsHome,
        percentageTotalWinHome, percentageHomeWinHome, percentageAwayWinHome,
        totalGamesAway, totalWinsAway, homeGamesAway, homeWinsAway, awayGamesAway, awayWinsAway,
        percentageTotalWinAway, percentageHomeWinAway, percentageAwayWinAway
    ]

# Add results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)
games[['teamHomeCode', 'teamAwayCode', 'percentageTotalWinHome', 'percentageHomeWinHome', 'percentageAwayWinHome', 'percentageTotalWinAway', 'percentageHomeWinAway', 'percentageAwayWinAway']].sample(5)

/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/jkentjnr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,teamHomeCode,teamAwayCode,percentageTotalWinHome,percentageHomeWinHome,percentageAwayWinHome,percentageTotalWinAway,percentageHomeWinAway,percentageAwayWinAway
341,TOR,PHX,0.692,0.808,0.577,0.316,0.3,0.333
328,OKL,SAS,0.577,0.667,0.48,0.589,0.778,0.414
431,BRO,IND,0.258,0.312,0.2,0.586,0.688,0.462
406,CLE,LAL,0.5,0.6,0.409,0.453,0.56,0.357
762,OKL,NOP,0.667,0.7,0.625,0.562,0.625,0.5


### Calculate the results for the last 5 games for the Home team.

In [34]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome'])

# Iterate through each game where the teams have played at least 5 games.
for index, game in games.iterrows():

    # Get the last five games for the team.
    last5games = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False).head(5)

    if len(last5games) == 5:
        lastGame1WinHome = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
        lastGame2WinHome = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
        lastGame3WinHome = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
        lastGame4WinHome = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
        lastGame5WinHome = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
        #print(lastGame1WinHome, lastGame2WinHome, lastGame3WinHome, lastGame4WinHome, lastGame5WinHome)

        lastGame1AtHomeHome = last5games["teamHomeCode"].iloc[0] == game.teamHomeCode
        lastGame2AtHomeHome = last5games["teamHomeCode"].iloc[1] == game.teamHomeCode
        lastGame3AtHomeHome = last5games["teamHomeCode"].iloc[2] == game.teamHomeCode
        lastGame4AtHomeHome = last5games["teamHomeCode"].iloc[3] == game.teamHomeCode
        lastGame5AtHomeHome = last5games["teamHomeCode"].iloc[4] == game.teamHomeCode
        #print(lastGame1AtHomeHome, lastGame2AtHomeHome, lastGame3AtHomeHome, lastGame4AtHomeHome, lastGame5AtHomeHome)

        # Update the row with the history
        data.loc[index] = [lastGame1WinHome, lastGame1AtHomeHome, lastGame2WinHome, lastGame2AtHomeHome, lastGame3WinHome, lastGame3AtHomeHome, lastGame4WinHome, lastGame4AtHomeHome, lastGame5WinHome, lastGame5AtHomeHome]

# Convert types 
data['lastGame1WinHome'] = data['lastGame1WinHome'].astype('bool')
data['lastGame2WinHome'] = data['lastGame2WinHome'].astype('bool')
data['lastGame3WinHome'] = data['lastGame3WinHome'].astype('bool')
data['lastGame4WinHome'] = data['lastGame4WinHome'].astype('bool')
data['lastGame5WinHome'] = data['lastGame5WinHome'].astype('bool')
data['lastGame1AtHomeHome'] = data['lastGame1AtHomeHome'].astype('bool')
data['lastGame2AtHomeHome'] = data['lastGame2AtHomeHome'].astype('bool')
data['lastGame3AtHomeHome'] = data['lastGame3AtHomeHome'].astype('bool')
data['lastGame4AtHomeHome'] = data['lastGame4AtHomeHome'].astype('bool')
data['lastGame5AtHomeHome'] = data['lastGame5AtHomeHome'].astype('bool')

# Add results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)

games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

,teamHomeCode,teamAwayCode,lastGame1WinHome,lastGame1AtHomeHome,lastGame2WinHome,lastGame2AtHomeHome,lastGame3WinHome,lastGame3AtHomeHome,lastGame4WinHome,lastGame4AtHomeHome,lastGame5WinHome,lastGame5AtHomeHome
647,CHI,GSW,True,True,True,True,False,True,False,False,True,False
175,MIL,LAL,True,False,True,False,False,False,True,False,False,True
890,NOP,PHX,True,True,True,True,False,True,True,True,True,False
90,BOS,SAS,True,True,False,False,False,False,False,True,True,True
351,NOP,DEN,True,False,False,True,True,False,False,False,True,False


### Calculate the results for the last 5 games for the Away team.

In [35]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(games)), columns=['lastGame1WinAway', 'lastGame1AtHomeAway', 'lastGame2WinAway', 'lastGame2AtHomeAway', 'lastGame3WinAway', 'lastGame3AtHomeAway', 'lastGame4WinAway', 'lastGame4AtHomeAway', 'lastGame5WinAway', 'lastGame5AtHomeAway'])

# Iterate through each game where the teams have played at least 5 games.
for index, game in games.iterrows():

    # Get the last five games for the team.
    last5games = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False).head(5)

    if len(last5games) == 5:
        lastGame1WinAway = last5games["scoreAway"].iloc[0] < last5games["scoreHome"].iloc[0]
        lastGame2WinAway = last5games["scoreAway"].iloc[1] < last5games["scoreHome"].iloc[1]
        lastGame3WinAway = last5games["scoreAway"].iloc[2] < last5games["scoreHome"].iloc[2]
        lastGame4WinAway = last5games["scoreAway"].iloc[3] < last5games["scoreHome"].iloc[3]
        lastGame5WinAway = last5games["scoreAway"].iloc[4] < last5games["scoreHome"].iloc[4]
        #print(lastGame1WinAway, lastGame2WinAway, lastGame3WinAway, lastGame4WinAway, lastGame5WinAway)

        lastGame1AtHomeAway = last5games["teamAwayCode"].iloc[0] == game.teamAwayCode
        lastGame2AtHomeAway = last5games["teamAwayCode"].iloc[1] == game.teamAwayCode
        lastGame3AtHomeAway = last5games["teamAwayCode"].iloc[2] == game.teamAwayCode
        lastGame4AtHomeAway = last5games["teamAwayCode"].iloc[3] == game.teamAwayCode
        lastGame5AtHomeAway = last5games["teamAwayCode"].iloc[4] == game.teamAwayCode
        #print(lastGame1AtHomeAway, lastGame2AtHomeAway, lastGame3AtHomeAway, lastGame4AtHomeAway, lastGame5AtHomeAway)

        # Update the row with the history
        data.loc[index] = [lastGame1WinAway, lastGame1AtHomeAway, lastGame2WinAway, lastGame2AtHomeAway, lastGame3WinAway, lastGame3AtHomeAway, lastGame4WinAway, lastGame4AtHomeAway, lastGame5WinAway, lastGame5AtHomeAway]

# Convert types         
data['lastGame1WinAway'] = data['lastGame1WinAway'].astype('bool')
data['lastGame2WinAway'] = data['lastGame2WinAway'].astype('bool')
data['lastGame3WinAway'] = data['lastGame3WinAway'].astype('bool')
data['lastGame4WinAway'] = data['lastGame4WinAway'].astype('bool')
data['lastGame5WinAway'] = data['lastGame5WinAway'].astype('bool')
data['lastGame1AtHomeAway'] = data['lastGame1AtHomeAway'].astype('bool')
data['lastGame2AtHomeAway'] = data['lastGame2AtHomeAway'].astype('bool')
data['lastGame3AtHomeAway'] = data['lastGame3AtHomeAway'].astype('bool')
data['lastGame4AtHomeAway'] = data['lastGame4AtHomeAway'].astype('bool')
data['lastGame5AtHomeAway'] = data['lastGame5AtHomeAway'].astype('bool')

# Add results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)

games[['teamHomeCode', 'teamAwayCode', 'lastGame1WinHome', 'lastGame1AtHomeHome', 'lastGame2WinHome', 'lastGame2AtHomeHome', 'lastGame3WinHome', 'lastGame3AtHomeHome', 'lastGame4WinHome', 'lastGame4AtHomeHome', 'lastGame5WinHome', 'lastGame5AtHomeHome']].sample(5)

,teamHomeCode,teamAwayCode,lastGame1WinHome,lastGame1AtHomeHome,lastGame2WinHome,lastGame2AtHomeHome,lastGame3WinHome,lastGame3AtHomeHome,lastGame4WinHome,lastGame4AtHomeHome,lastGame5WinHome,lastGame5AtHomeHome
617,LAC,SAC,False,False,True,True,False,False,False,True,False,True
407,MIN,SAC,False,False,False,True,True,True,True,True,False,False
507,MIL,MIN,True,True,True,True,True,True,True,True,True,True
308,POR,MIL,False,False,False,False,False,False,True,False,True,True
202,CHA,CLE,True,False,True,False,True,False,True,False,True,True


### Calculate the offensive & defensive rating for the last 5 games

In [36]:
# Create a dataframe for the results - same size as dataset
data = pd.DataFrame(index=range(0,len(games)), columns=[
    'calcStatsHome.seasonTotalPossessions', 'calcStatsHome.last5TotalPossessions',
    'calcStatsHome.seasonTotalPoints', 'calcStatsHome.last5TotalPoints',
    'calcStatsHome.seasonOffensiveRating', 'calcStatsHome.last5OffensiveRating',
    'calcStatsHome.seasonDefensiveRating', 'calcStatsHome.last5DefensiveRating',
    'calcStatsAway.seasonTotalPossessions', 'calcStatsAway.last5TotalPossessions',
    'calcStatsAway.seasonTotalPoints', 'calcStatsAway.last5TotalPoints',
    'calcStatsAway.seasonOffensiveRating', 'calcStatsAway.last5OffensiveRating',
    'calcStatsAway.seasonDefensiveRating', 'calcStatsAway.last5DefensiveRating'
])

# Iterate through each game where the teams have played at least 5 games.
for index, game in games.iterrows():

    # Get all previous games - Home
    allgames = games[(games['season'] == game.season) & (games['gamesPlayedHome'] < game.gamesPlayedHome) & ((games['teamAwayCode'] == game.teamHomeCode) | (games['teamHomeCode'] == game.teamHomeCode))].sort_values(by='gamesPlayedHome', ascending=False)
    
    totalPossessionsHome = allgames['calcStatsHome.FgAtt'].sum() - allgames['calcStatsHome.OffReb'].sum() + allgames['calcStatsHome.Tov'].sum() + (0.4 * allgames['calcStatsHome.FtAtt'].sum());
    last5PossessionsHome = allgames['calcStatsHome.FgAtt'].head(5).sum() - allgames['calcStatsHome.OffReb'].head(5).sum() + allgames['calcStatsHome.Tov'].head(5).sum() + (0.4 * allgames['calcStatsHome.FtAtt'].head(5).sum());

    totalPointsScoredHome = allgames['calcStatsHome.Pts'].sum()
    last5PointsScoredHome = allgames['calcStatsHome.Pts'].head(5).sum()

    totalOffensiveRatingHome = round(calc_percentage(totalPointsScoredHome, totalPossessionsHome) * 100, 2)
    last5OffensiveRatingHome = round(calc_percentage(last5PointsScoredHome, last5PossessionsHome) * 100, 2)

    totalPointsAgainstHome = allgames['calcStatsHome.PtsAgainst'].sum()
    last5PointsAgainstHome = allgames['calcStatsHome.PtsAgainst'].head(5).sum()

    totalDefensiveRatingHome = round(calc_percentage(totalPointsAgainstHome, totalPossessionsHome) * 100, 2)
    last5DefensiveRatingHome = round(calc_percentage(last5PointsAgainstHome, last5PossessionsHome) * 100, 2)

    # Get all previous games - Away
    allgames = games[(games['season'] == game.season) & (games['gamesPlayedAway'] < game.gamesPlayedAway) & ((games['teamAwayCode'] == game.teamAwayCode) | (games['teamHomeCode'] == game.teamAwayCode))].sort_values(by='gamesPlayedAway', ascending=False)

    totalPossessionsAway = allgames['calcStatsHome.FgAtt'].sum() - allgames['calcStatsHome.OffReb'].sum() + allgames['calcStatsHome.Tov'].sum() + (0.4 * allgames['calcStatsHome.FtAtt'].sum());
    last5PossessionsAway = allgames['calcStatsHome.FgAtt'].head(5).sum() - allgames['calcStatsHome.OffReb'].head(5).sum() + allgames['calcStatsHome.Tov'].head(5).sum() + (0.4 * allgames['calcStatsHome.FtAtt'].head(5).sum());

    totalPointsScoredAway = allgames['calcStatsHome.Pts'].sum()
    last5PointsScoredAway = allgames['calcStatsHome.Pts'].head(5).sum()

    totalOffensiveRatingAway = round(calc_percentage(totalPointsScoredAway, totalPossessionsAway) * 100, 2)
    last5OffensiveRatingAway = round(calc_percentage(last5PointsScoredAway, last5PossessionsAway) * 100, 2)

    totalPointsAgainstAway = allgames['calcStatsHome.PtsAgainst'].sum()
    last5PointsAgainstAway = allgames['calcStatsHome.PtsAgainst'].head(5).sum()

    totalDefensiveRatingAway = round(calc_percentage(totalPointsAgainstAway, totalPossessionsAway) * 100, 2)
    last5DefensiveRatingAway = round(calc_percentage(last5PointsAgainstAway, last5PossessionsAway) * 100, 2)

    data.loc[index] = [
        totalPossessionsHome, last5PossessionsHome, 
        totalPointsScoredHome, last5PointsScoredHome, 
        totalOffensiveRatingHome, last5OffensiveRatingHome,
        totalDefensiveRatingHome, last5DefensiveRatingHome,
        totalPossessionsAway, last5PossessionsAway, 
        totalPointsScoredAway, last5PointsScoredAway, 
        totalOffensiveRatingAway, last5OffensiveRatingAway,
        totalDefensiveRatingAway, last5DefensiveRatingAway
    ]

# Add results to the dataset
games = pd.merge(games, data, left_index=True, right_index=True)

In [37]:
# test entry for stadium distance calc

coords_1 = (37.750267, -122.202853)
coords_2 = (34.04303865743706, -118.26711416244507)

geopy.distance.vincenty(coords_1, coords_2).miles


337.6783477035334

### Convert the home and away teams to one-hot encoding 

In [38]:
if use_team_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='team')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='team')

    homeTeam = homeTeam.replace({0:np.nan})
    awayTeam = awayTeam.replace({0:np.nan})

    teams = homeTeam.fillna(awayTeam).fillna(0).astype('bool')
    
    games = pd.concat([games, teams], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'team_NYK', 'team_CLE']].head(2)

In [39]:
if use_team_home_away_hot_encoding == True:
    homeTeam = pd.get_dummies(games["teamHomeCode"], prefix='hometeam').astype('bool')
    awayTeam = pd.get_dummies(games["teamAwayCode"], prefix='awayteam').astype('bool')

    games = pd.concat([games, homeTeam, awayTeam], axis=1)
    
    games[['teamHomeCode', 'teamAwayCode', 'hometeam_NYK', 'hometeam_CLE', 'awayteam_NYK', 'awayteam_CLE']].head(2)

### Drop the id fields for teams.

In [40]:
games = games.drop(["teamAwayId", "teamHomeId"], axis=1)

In [41]:
#gamesQuarters = json_normalize(data=d['games'], record_path='scoreQuarters', meta=['id'])
#gamesQuarters.head(10)

### Determine winner flag

In [42]:
games["homeWin"] = games["scoreHome"] > games["scoreAway"]

## Save data to CSV to save on processing

In [43]:
games.to_csv(output_parsed_data_filename, encoding='utf-8')